In [1]:
# from bokeh.sampledata.gapminder import population, fertility, life_expectancy
import bokeh.sampledata
bokeh.sampledata.download()
from bokeh.sampledata.gapminder import population, fertility, life_expectancy

Using data directory: C:\Users\rosme\.bokeh\data
Skipping 'CGM.csv' (checksum match)
Skipping 'US_Counties.zip' (checksum match)
Skipping 'us_cities.json' (checksum match)
Skipping 'unemployment09.csv' (checksum match)
Skipping 'AAPL.csv' (checksum match)
Skipping 'FB.csv' (checksum match)
Skipping 'GOOG.csv' (checksum match)
Skipping 'IBM.csv' (checksum match)
Skipping 'MSFT.csv' (checksum match)
Skipping 'WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip' (checksum match)
Skipping 'gapminder_fertility.csv' (checksum match)
Skipping 'gapminder_population.csv' (checksum match)
Skipping 'gapminder_life_expectancy.csv' (checksum match)
Skipping 'gapminder_regions.csv' (checksum match)
Skipping 'world_cities.zip' (checksum match)
Skipping 'airports.json' (checksum match)
Skipping 'movies.db.zip' (checksum match)
Skipping 'airports.csv' (checksum match)
Skipping 'routes.csv' (checksum match)
Skipping 'haarcascade_frontalface_default.xml' (checksum match)


In [2]:
## bokeh_animation_ex1.py

from bokeh.models import Button, Slider
from bokeh.plotting import figure, column, row
from bokeh.models.ranges import FactorRange
from bokeh.io import curdoc

import time

## Prepare Data
def convert_population(population):
    if population > 1e9:
        pop = "{:.2f} Bn".format(population / 1e9)
    else:
        pop = "{:.2f} Mn".format(population / 1e6)
    return pop

top_10 = population[["1964"]].dropna().sort_values(by=["1964"], ascending=False).reset_index()
top_10 = top_10.rename(columns={"1964": "Population"}).head(10).reset_index()
top_10["index"] = top_10["index"] + 1
top_10["index"] = top_10["index"].values[::-1]
top_10 = top_10.sort_values(by="Population")
top_10["Population_Text"] = top_10["Population"].apply(convert_population)


## Create Chart
fig = figure(width=700, height=500, title="Top 10 Countries by Population (1964-2013)",
             x_range=(0, 1.6e9))

bars = fig.hbar(y="index", right="Population", color="dodgerblue",
                height=0.85, source=top_10)
text = fig.text(x="Population", y="index", text="Population_Text", x_offset=5,
                text_baseline="middle", source=top_10)

x_range = list(map(int, [0, 200e6, 400e6, 600e6, 800e6, 1e9, 1.2e9, 1.4e9, 1.6e9]))
x_range_ticks = ["1 Mn", "200 Mn", "400 Mn", "600 Mn", "800 Mn", "1.00 Bn", "1.20 Bn", "1.40 Bn", "1.6 Bn"]
#fig.xaxis.ticker = x_range
fig.xaxis.major_label_overrides = dict(zip(x_range, x_range_ticks))

fig.yaxis.ticker = list(range(1,11))
fig.yaxis.major_label_overrides = dict(zip(list(range(1,11)), top_10.Country.values))

## Define Widgets
slider = Slider(start=1964, end=2013, value=1964, step=1, title="Year:")
btn = Button(label="Play")

## Define Callbacks
curr_year = 1964
def update_chart():
    global curr_year
    curr_year += 1
    slider.value = curr_year
    if curr_year == 2013:
        curr_year = 1964

callback = None
def execute_animation():
    global callback
    if btn.label == "Play":
        btn.label = "Pause"
        callback = curdoc().add_periodic_callback(update_chart, 100)
    else:
        btn.label = "Play"
        curdoc().remove_periodic_callback(callback)


def modify_chart(attr, old, new):
    idx = "{}".format(slider.value)
    top_10 = population[[idx]].dropna().sort_values(by=[idx], ascending=False).reset_index()
    top_10 = top_10.rename(columns={idx: "Population"}).head(10).reset_index()
    top_10["index"] = top_10["index"] + 1
    top_10["index"] = top_10["index"].values[::-1]
    top_10 = top_10.sort_values(by="Population")
    top_10["Population_Text"] = top_10["Population"].apply(convert_population)

    bars.data_source.data = top_10
    text.data_source.data = top_10

    fig.yaxis.major_label_overrides = dict(zip(list(range(1,10)), top_10.Country.values))


## Register Callbacks
btn.on_click(execute_animation)
slider.on_change("value", modify_chart)


## GUI
curdoc().add_root(column(btn, slider, fig))

In [3]:
!bokeh serve --show --port 2009 poblacion.ipynb

^C
